In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np


Download dữ liệu chữ số viết tay MNIST

In [ ]:
# Chuẩn bị dữ liệu
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Chuyển đổi sang định dạng float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Chuẩn hóa ảnh từ from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.
x_train, x_test, y_train, y_test = torch.from_numpy(x_train), torch.from_numpy(x_test), torch.from_numpy(y_train).type(torch.LongTensor), torch.from_numpy(y_test).type(torch.LongTensor)

In [ ]:
batch_size = 16
trainloader = []
for (i,j) in zip(x_train, y_train):
    trainloader.append([i,j])
trainloader = torch.utils.data.DataLoader(trainloader, shuffle=True, batch_size=batch_size)

testloader = []
for (i,j) in zip(x_test, y_test):
    testloader.append([i,j])
testloader = torch.utils.data.DataLoader(testloader, shuffle=True, batch_size=batch_size)


In [ ]:
num_features = 784
n_hidden_1 = 512
n_hidden_2 = 128
n_hidden_3 = 32
num_classes = 10

Sử dụng các tham số ở trên để xây dựng mô hình

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_features, n_hidden_1)
        self.fc2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.fc3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.fc4 = nn.Linear(n_hidden_3, num_classes)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # load input và labels
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # print(labels.shape)
        # print(outputs.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.540
[2,  2000] loss: 0.329
[3,  2000] loss: 0.209
[4,  2000] loss: 0.151
[5,  2000] loss: 0.119
[6,  2000] loss: 0.100
[7,  2000] loss: 0.080
[8,  2000] loss: 0.067
[9,  2000] loss: 0.056
[10,  2000] loss: 0.046
[11,  2000] loss: 0.040
[12,  2000] loss: 0.031
[13,  2000] loss: 0.028
[14,  2000] loss: 0.023
[15,  2000] loss: 0.019
[16,  2000] loss: 0.015
[17,  2000] loss: 0.013
[18,  2000] loss: 0.012
[19,  2000] loss: 0.007
[20,  2000] loss: 0.007
Finished Training


In [ ]:
correct = 0
total = 0
# do đang thực hiện việc dự đoán nên ko cần tính đạo hàm
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        # chạy hàm dự đoán
        outputs = net(inputs)
        # the class với giá trị xác suất cao nhất là đâu ra dự đoán
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %


## Save and load model<br>
Trình bày 1 trong các các lưu model và load model trong PyTorch

1.   Lưu model<br>
```
torch.save(model.state_dict(), PATH)
```
trong đó PATH là đường dẫn tự định nghĩa
2.   Load model <br>

*   Trước tiên phải định nghĩa model trước. Model được định nghĩa phải giống hệt với model đã được lưu lại. Như ví dụ trong bài này, thì sẽ thực hiện như sau:
```
model = Net()
```
*   Load trọng số đã được học vào mô hình<br>
```
model.load_state_dict(torch.load(PATH))
# vô hiệu hóa các layer như Dropout hay BatchNorm
model.eval()
```
3. Có thể tham khảo thêm các phương pháp lưu và load model tại: https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html






